In [1]:
#dd-ignore

from dd_scenario import *

#Creates a client...
client = Client()


In [2]:
#dd-ignore

#Get 'TerritoryAssignment' decision...
dd_model_builder = client.get_model_builder(name="TerritoryAssignment")

#Get scenario 'Scenario 1'...
scenario = dd_model_builder.get_scenario(name="Scenario 1")

#Load all input data as a map { data_name: data_frame }
inputs = scenario.get_tables_data(category='input')
# This will hold all outputs as a map { data_name: data_frame }
outputs = {}

# we use a lock to access ``outputs``. This allows solves() to
# be aborted without race condition in data writting
import threading
output_lock = threading.Lock()



In [7]:
from docplex.mp.model import *
from docloud.status import JobSolveStatus
from docplex.mp.conflict_refiner import ConflictRefiner, VarUbConstraintWrapper, VarLbConstraintWrapper
import time
import sys

import pandas as pd
import numpy as np


# Label constraint
def helper_add_labeled_cplex_constraint(mdl, expr, label, context=None, columns=None):
    global expr_counter
    if isinstance(expr, bool):
        pass  # Adding a trivial constraint: if infeasible, docplex will raise an exception it is added to the model
    else:
        expr.name = '_L_EXPR_' + str(len(expr_to_info) + 1)
        if columns:
            ctxt = ", ".join(str(getattr(context, col)) for col in columns)
        else:
            if context:
                ctxt = context.Index if isinstance(context.Index, str) is not None else ", ".join(context.Index)
            else:
                ctxt = None
        expr_to_info[expr.name] = (label, ctxt)
    mdl.add(expr)



# Data model definition for each table
# Data collection: list_of_Salesrep ['capacity', 'home', 'salesrep']
# Data collection: list_of_State ['customers', 'state']

# Create a pandas Dataframe for each data table
list_of_Salesrep = inputs['salesrep']
list_of_Salesrep = list_of_Salesrep[['capacity', 'home', 'salesrep']].copy()
list_of_Salesrep.rename(columns={'capacity': 'capacity', 'home': 'home', 'salesrep': 'salesrep'}, inplace=True)
list_of_State = inputs['state']
list_of_State = list_of_State[['customers', 'state']].copy()
list_of_State.rename(columns={'customers': 'customers', 'state': 'state'}, inplace=True)

# Set index when a primary key is defined
list_of_Salesrep.set_index('salesrep', inplace=True)
list_of_Salesrep.sort_index(inplace=True)
list_of_Salesrep.index.name = 'id_of_Salesrep'
list_of_State.set_index('state', inplace=True)
list_of_State.sort_index(inplace=True)
list_of_State.index.name = 'id_of_State'

# Create data frame as cartesian product of: Salesrep x State
list_of_ResourceAssignment = pd.DataFrame(index=pd.MultiIndex.from_product((list_of_Salesrep.index, list_of_State.index), names=['id_of_Salesrep', 'id_of_State']))

# distance table for distance from cResourceAssignment[salesrep, state] / state / state to cResourceAssignment[salesrep, state] / salesrep / home / state
#list_of_Distancestatestate = pd.read_csv('list_of_Distancestatestate_data.csv')
list_of_Distancestatestate = inputs['distance']
#list_of_Distancestatestate.set_index(['fromLocation', 'toLocation'], inplace=True)
#list_of_Distancestatestate.set_index(['from', 'to'], inplace=True)


def build_model():
    mdl = Model()

    # Definition of model variables
    list_of_ResourceAssignment['resourceAssignmentVar'] = mdl.binary_var_list(len(list_of_ResourceAssignment))


    # Definition of model
    # Objective cMaximizeAssignmentsAutoSelected-
    # Combine weighted criteria: 
    # 	cMaximizeAssignmentsAutoSelected cMaximizeAssignmentsAutoSelected{
    # 	cScaledGoal.scaleFactorExpr = 1,
    # 	cSingleCriterionGoal.goalFilter = null,
    # 	cSingleCriterionGoal.numericExpr = count( cResourceAssignment[salesrep, state]),
    # 	cMaximizeAssignments.assignment = cResourceAssignment[salesrep, state]} with weight 5.0
    # 	cMinimizeGoalAssign cMinimizeGoalAssign{
    # 	cScaledGoal.scaleFactorExpr = 1,
    # 	cSingleCriterionGoal.goalFilter = null,
    # 	cSingleCriterionGoal.numericExpr = distance from cResourceAssignment[salesrep, state] / state / state to cResourceAssignment[salesrep, state] / salesrep / home / state} with weight 5.0
    agg_ResourceAssignment_resourceAssignmentVar_SG1 = mdl.sum(list_of_ResourceAssignment.resourceAssignmentVar)
    join_ResourceAssignment_Salesrep_SG2 = list_of_ResourceAssignment.join(list_of_Salesrep.home, how='inner')
    join_ResourceAssignment_Salesrep_list_of_Distancestatestate_SG2 = join_ResourceAssignment_Salesrep_SG2.reset_index().join(list_of_Distancestatestate.value, on=['id_of_State', 'home'], how='inner').set_index(join_ResourceAssignment_Salesrep_SG2.index.names)
    join_ResourceAssignment_Salesrep_list_of_Distancestatestate_SG2['conditioned_value'] = join_ResourceAssignment_Salesrep_list_of_Distancestatestate_SG2.resourceAssignmentVar * join_ResourceAssignment_Salesrep_list_of_Distancestatestate_SG2.value
    
    mdl.add_kpi(16.0 * (agg_ResourceAssignment_resourceAssignmentVar_SG1) / 1, publish_name='the number of salesrep to state assignments')
    mdl.add_kpi(float(16.0) * (join_ResourceAssignment_Salesrep_list_of_Distancestatestate_SG2) / 1, publish_name='distance between assigned states and home of assigned salesreps')
    
    mdl.minimize( 0
        # Sub Goal cMaximizeAssignmentsAutoSelected_cMaximizeGoal
        # Maximize the number of salesrep to state assignments
        - 16.0 * (agg_ResourceAssignment_resourceAssignmentVar_SG1) / 1
        # Sub Goal cMinimizeGoalAssign_cMinimizeGoal
        # Minimize distance between assigned states and home of assigned salesreps
        + 16.0 * (join_ResourceAssignment_Salesrep_list_of_Distancestatestate_SG2) / 1
    )
    
    # [ST_1] Constraint : cLimitNumberOfResourcesAssignedToEachActivity_cIterativeRelationalConstraint
    # The number of salesrep assignments for each state is less than or equal to 1
    # Label: CT_1_The_number_of_salesrep_assignments_for_each_state_is_less_than_or_equal_to_1
    join_State_ResourceAssignment = list_of_State.join(list_of_ResourceAssignment, how='inner')
    groupbyLevels = [join_State_ResourceAssignment.index.names.index(name) for name in list_of_State.index.names]
    groupby_State_ResourceAssignment = join_State_ResourceAssignment.resourceAssignmentVar.groupby(level=groupbyLevels).sum().to_frame()
    for row in groupby_State_ResourceAssignment.itertuples(index=True):
        helper_add_labeled_cplex_constraint(mdl, row.resourceAssignmentVar <= 1, 'The number of salesrep assignments for each state is less than or equal to 1', row)
    
    # [ST_2] Constraint : cResourceRelationalConstraint_cIterativeRelationalConstraint
    # For each salesrep, total customers of assigned states is less than capacity
    # Label: CT_2_For_each_salesrep__total_customers_of_assigned_states_is_less_than_capacity
    join_Salesrep_ResourceAssignment = list_of_Salesrep.join(list_of_ResourceAssignment, how='inner')
    join_Salesrep_ResourceAssignment_State = join_Salesrep_ResourceAssignment.join(list_of_State.customers, how='inner')
    join_Salesrep_ResourceAssignment_State['conditioned_customers'] = join_Salesrep_ResourceAssignment_State.resourceAssignmentVar * join_Salesrep_ResourceAssignment_State.customers
    groupbyLevels = [join_Salesrep_ResourceAssignment_State.index.names.index(name) for name in list_of_Salesrep.index.names]
    groupby_Salesrep_ResourceAssignment_State = join_Salesrep_ResourceAssignment_State.conditioned_customers.groupby(level=groupbyLevels).sum().to_frame()
    join_Salesrep_ResourceAssignment_State_Salesrep = groupby_Salesrep_ResourceAssignment_State.join(list_of_Salesrep.capacity, how='inner')
    for row in join_Salesrep_ResourceAssignment_State_Salesrep[join_Salesrep_ResourceAssignment_State_Salesrep.capacity.notnull()].itertuples(index=True):
        helper_add_labeled_cplex_constraint(mdl, row.conditioned_customers <= -0.001 + row.capacity, 'For each salesrep, total customers of assigned states is less than capacity', row)
    
    # [ST_3] Constraint : cManualAssignment_cCategoryCompatibilityConstraintOnPair
    # California is assigned to John
    # Label: CT_3_California_is_assigned_to_John
    filtered_Salesrep = list_of_Salesrep.loc[['John']]
    join_Salesrep_ResourceAssignment = filtered_Salesrep.join(list_of_ResourceAssignment, how='inner')
    filtered_State = list_of_State.loc[['California']]
    join_Salesrep_ResourceAssignment_State = join_Salesrep_ResourceAssignment.reset_index().join(filtered_State, on=['id_of_State'], how='inner').set_index(join_Salesrep_ResourceAssignment.index.names)
    helper_add_labeled_cplex_constraint(mdl, mdl.sum(join_Salesrep_ResourceAssignment_State.resourceAssignmentVar) >= 1, 'California is assigned to John')
    
    # [ST_4] Constraint : cBasicLimitNumberOfActivitiesAssignedToEachResource_cIterativeRelationalConstraint
    # The number of state assignments for each salesrep is less than or equal to 1
    # Label: CT_4_The_number_of_state_assignments_for_each_salesrep_is_less_than_or_equal_to_1
    join_Salesrep_ResourceAssignment = list_of_Salesrep.join(list_of_ResourceAssignment, how='inner')
    groupbyLevels = [join_Salesrep_ResourceAssignment.index.names.index(name) for name in list_of_Salesrep.index.names]
    groupby_Salesrep_ResourceAssignment = join_Salesrep_ResourceAssignment.resourceAssignmentVar.groupby(level=groupbyLevels).sum().to_frame()
    for row in groupby_Salesrep_ResourceAssignment.itertuples(index=True):
        helper_add_labeled_cplex_constraint(mdl, row.resourceAssignmentVar <= 1, 'The number of state assignments for each salesrep is less than or equal to 1', row)


    return mdl


def solve_model(mdl):
    mdl.parameters.timelimit = 120
    msol = mdl.solve(log_output=True)
    if not msol:
        print("!!! Solve of the model fails")
        if mdl.get_solve_status() == JobSolveStatus.INFEASIBLE_SOLUTION:
            crefiner = ConflictRefiner()
            conflicts = crefiner.refine_conflict(model, log_output=True)
            export_conflicts(conflicts)
    print 'Solve status: ', mdl.get_solve_status()
    mdl.report()
    return msol


expr_to_info = {}


def export_conflicts(conflicts):
    # Display conflicts in console
    print('Conflict set:')
    list_of_conflicts = pd.DataFrame(columns=['constraint', 'context', 'detail'])
    for conflict, index in zip(conflicts, range(len(conflicts))):
        st = conflict.status
        ct = conflict.element
        label, context = expr_to_info.get(conflict.name, ('N/A', conflict.name))
        label_type = type(conflict.element)
        if isinstance(conflict.element, VarLbConstraintWrapper) \
                or isinstance(conflict.element, VarUbConstraintWrapper):
            ct = conflict.element.get_constraint()

        # Print conflict information in console
        print("Conflict involving constraint: %s, \tfor: %s -> %s" % (label, context, ct))
        list_of_conflicts = list_of_conflicts.append(pd.DataFrame({'constraint': label, 'context': str(context), 'detail': ct},
                                                                  index=[index], columns=['constraint', 'context', 'detail']))

    # Update of the ``outputs`` dict must take the 'Lock' to make this action atomic,
    # in case the job is aborted
    global output_lock
    with output_lock:
        outputs['list_of_conflicts'] = list_of_conflicts


def export_solution(msol):
    start_time = time.time()
    list_of_ResourceAssignment_solution = pd.DataFrame(index=list_of_ResourceAssignment.index)
    list_of_ResourceAssignment_solution['resourceAssignmentVar'] = msol.get_values(list_of_ResourceAssignment.resourceAssignmentVar.values)

    # Filter rows for non-selected assignments
    list_of_ResourceAssignment_solution = list_of_ResourceAssignment_solution[list_of_ResourceAssignment_solution.resourceAssignmentVar > 0.5]

    # Update of the ``outputs`` dict must take the 'Lock' to make this action atomic,
    # in case the job is aborted
    global output_lock
    with output_lock:
        outputs['list_of_ResourceAssignment_solution'] = list_of_ResourceAssignment_solution.reset_index()

    elapsed_time = time.time() - start_time
    print('solution export done in ' + str(elapsed_time) + ' secs')
    return


print('* building wado model')
start_time = time.time()
model = build_model()
elapsed_time = time.time() - start_time
print('model building done in ' + str(elapsed_time) + ' secs')

print('* running wado model')
start_time = time.time()
msol = solve_model(model)
elapsed_time = time.time() - start_time
print('model solve done in ' + str(elapsed_time) + ' secs')
if msol:
    export_solution(msol)

* building wado model


TypeError: Could not operate 16.0 with block values can't multiply sequence by non-int of type 'float'